In [25]:
import polars.selectors as cs
import numpy as np

import polars as pl

import matplotlib.pyplot as plt

In [26]:
def hist_of_numerical_values(df: pl.DataFrame):
    """
    Generates histograms for all numerical columns in a Polars DataFrame.

    Args:
        df: The Polars DataFrame.

    Returns:
        A dictionary where keys are column names and values are the histogram data.
    """
    histograms = {}
    for col_name in df.columns:
        if df[col_name].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
            hist_data = df[col_name].hist()
            histograms[col_name] = hist_data
    return histograms

In [27]:
index_cols = {
    1: ['l_returnflag', 'l_linestatus'],
    3: ['l_orderkey','o_orderdate','o_shippriority'],
    4: ['o_orderpriority'],
    5: ['n_name'],
    6: [],
    7: ['supp_nation', 'cust_nation', 'l_year'], 
    8: ['o_year'],
    9: ['nation', 'o_year'],
    12: ['l_shipmode'],
    13: ['c_count'],
    14: [],
    15: [],
    17: [],
    19: [],
    20: [],
    21: ['s_name'],
    22: ['cntrycode']
}
scale_required = set([1,4, 5, 6, 7, 9, 12, 13, 17, 19, 21, 22])
# not 8, 14
# skip 3, skip 20
queries_to_run = [1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 17, 19, 21, 22]

In [32]:
# for query_ind in queries_to_run:
num_trials = 1000
avg_errors = {}
all_nulls = {}
for query_ind in queries_to_run:
#     print(query_ind)
#     1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1.0, 2.0, 4.0
    num_nulls = 0.
    total_error = []
    for mi in [1/16]:
        null_info = {}
        orig = pl.read_csv(f'../unnoised/q{query_ind}.csv')
        if query_ind == 22:
            orig = orig.select(
            pl.col("cntrycode").cast(str),
            pl.col("numcust"),
            pl.col("totacctbal"))
        noised = pl.read_json(f'../outputs/ap-duckdb-q{query_ind}-customer-{mi}-step3/output.json')
        if index_cols[query_ind]:
            merged_df = orig.join(noised, on=index_cols[query_ind], suffix='_noised')
        else:
            merged_df = orig.join(noised, suffix='_noised', how='cross')
        suffix1 = ''
        suffix2 = '_noised'
        cols_with_suffixes = [col for col in merged_df.columns if suffix1 in col or suffix2 in col]

        base_names = set([col.replace(suffix1, '').replace(suffix2, '') for col in cols_with_suffixes])
        if query_ind == 15:
            base_names = ['total_revenue'] # only look at numeric
        all_outputs = 0.
        for base_name in base_names:
            nulls_exist = False
            null_inds, null_vals = [], []

            orig = base_name + suffix1
            noised = base_name + suffix2
            if orig in merged_df.columns and noised in merged_df.columns:
                rel_errors = []
                for ind in range(len(merged_df[orig])):
                    if query_ind in scale_required:
                        const = 2 # subsampling scaling
                        if query_ind == 1 and not (base_name[:3] == 'sum' or base_name[:5] == 'count'):
                            const = 1 # remove for parts of q1
                    else:
                        const = 1
    #                     print(merged_df[noised][ind])
                    noised_vals = [merged_df[noised][ind][tmp_ind] for tmp_ind in range(
                        len(merged_df[noised][ind])) if merged_df[noised][ind][tmp_ind] is not None]
                    if len(noised_vals) != 1000 and len(noised_vals) != 0:
                        print('reached', query_ind)
                    new = np.average(
                        [100*abs(
                            const*noised_vals[tmp_ind] - merged_df[orig][ind]
                            ) / merged_df[orig][ind] for tmp_ind in range(
                                len(noised_vals))]
                    )
                    if not np.isnan(new):
                        total_error.append(new)
                    rel_errors.append(new)
                merged_df = merged_df.with_columns(pl.Series('pct_diff_' + base_name, rel_errors))
                num_outputs = len(list(merged_df[noised]))
                all_outputs+= num_outputs
                for ind in range(num_outputs):
                    full_list = [x for x in list(merged_df[noised][ind]) if x is not None]
                    if len(full_list) < num_trials:
                        print(full_list, noised, ind, query_ind)
                        print('reached')
                        num_nulls += 1
                        nulls_exist = True
                        null_inds.append(ind)
                        null_vals.append(100*(num_trials-len(full_list))/num_trials)
#                         print(query_ind, ind, null_vals)
            null_info[base_name] = (nulls_exist, null_inds, null_vals)
        all_nulls[query_ind] = 100*num_nulls/all_outputs
        avg_errors[query_ind] = np.average(total_error)
        print(query_ind, len(total_error))

#         # Get columns that start with 'pct_diff'
#         pct_diff_cols = [col for col in merged_df.columns if col.startswith('pct_diff')]

#         # Create subplots based on number of pct_diff columns
#         n_cols = len(pct_diff_cols)
#         fig, axes = plt.subplots(1, n_cols, figsize=(5*n_cols, 4))

#         # Handle case where there's only one column (axes won't be a list)
#         if n_cols == 1:
#             axes = [axes]

#         # Create histogram for each column
#         for i, col in enumerate(pct_diff_cols):
#             # Convert to pandas for easier plotting, or use polars plot method
#             values = merged_df[col].to_pandas()

#             axes[i].hist(values, bins=30, alpha=0.7, edgecolor='black')
#             axes[i].set_title(f'Histogram of {col}')
#             axes[i].set_xlabel(col)
#             axes[i].set_ylabel('Frequency')
#             axes[i].grid(True, alpha=0.3)

#         plt.tight_layout()
#         plt.savefig(f'tmp/q{query_ind}_{mi}_{col}_hist.png', facecolor='white', bbox_inches='tight')

#         for base in null_info:
#             (nulls_exist, null_inds, null_vals) = null_info[base]
#             if nulls_exist:
#                 fig, axes = plt.subplots(1, 1, figsize=(5, 4))
#                 # Handle case where there's only one column (axes won't be a list)
#                 if n_cols == 1:
#                     axes = [axes]
#                 axes[0].hist(null_vals, bins=30, alpha=0.7, edgecolor='black')
#                 axes[0].set_title(f'Histogram of nulls for column {base}')
#                 axes[0].set_xlabel('Fraction of nulls (%)')
#                 axes[0].set_ylabel('Frequency')
#                 axes[0].grid(True, alpha=0.3)
#                 plt.savefig(f'tmp/q{query_ind}_{mi}_{base}_nulls.png', facecolor='white', bbox_inches='tight')

1 32
[] revenue_noised 0 3
reached
[] revenue_noised 1 3
reached
[] revenue_noised 2 3
reached
[] revenue_noised 3 3
reached
[] revenue_noised 4 3
reached
[] revenue_noised 5 3
reached
[] revenue_noised 6 3
reached
[] revenue_noised 7 3
reached
[] revenue_noised 8 3
reached
[] revenue_noised 9 3
reached
3 0
4 5
5 5
6 1
7 4
8 2
9 175
12 4
[] custdist_noised 37 13
reached
[] custdist_noised 38 13
reached
[] custdist_noised 39 13
reached
[] custdist_noised 40 13
reached
[] custdist_noised 41 13
reached
13 37
14 1
15 1
17 1
19 1
[] numwait_noised 86 21
reached
21 99
22 14


In [33]:
all_nulls

{1: 0.0,
 3: 100.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 12: 0.0,
 13: 11.904761904761905,
 14: 0.0,
 15: 0.0,
 17: 0.0,
 19: 0.0,
 21: 1.0,
 22: 0.0}

In [34]:
avg_errors

{1: 0.7556880802111988,
 3: nan,
 4: 2.4040037081433274,
 5: 8.854581686905387,
 6: 1.1866798958039402,
 7: 8.780564309520603,
 8: 35.54604282613709,
 9: 7.068615806325052,
 12: 3.0503490410683853,
 13: 10.854701901760366,
 14: 2.0583587986616276,
 15: 35.747335576100014,
 17: 11.474393040914164,
 19: 24.275938584227244,
 21: 63.8807483675156,
 22: 7.744830183693608}

In [35]:
num_nulls

0.0

In [24]:
merged_df

cntrycode,numcust,totacctbal,numcust_noised,totacctbal_noised,pct_diff_numcust,pct_diff_totacctbal
str,i64,f64,list[f64],list[f64],f64,f64
"""13""",888,6.7377e6,"[441.831781, 474.324589, … 363.347371]","[3.5190e6, 3.1232e6, … 3.5377e6]",7.993566,7.746325
"""17""",861,6.4606e6,"[357.548579, 415.566816, … 422.546507]","[2.9259e6, 2.8198e6, … 2.8126e6]",8.055456,8.083365
"""18""",964,7236687.4,"[486.820084, 461.719245, … 523.528831]","[3.3549e6, 3.8696e6, … 4.0587e6]",7.491968,7.731667
"""23""",892,6.7015e6,"[403.227495, 463.310648, … 524.278765]","[3.1450e6, 3.5901e6, … 3.7229e6]",7.596054,7.720936
"""29""",948,7.1589e6,"[585.327225, 479.463366, … 434.931105]","[3.3869e6, 3.8867e6, … 3.7268e6]",7.943,8.300938
"""30""",909,6.8084e6,"[414.385858, 473.187867, … 463.357404]","[3.3294e6, 3.9569e6, … 3.3951e6]",7.80886,8.363162
"""31""",922,6.8067e6,"[489.855638, 490.730762, … 448.345043]","[3.8486e6, 4.0927e6, … 3.0329e6]",7.760581,8.128501
